In [18]:
from openeye.oechem import *
from openeye.oeshape import *
import glob
import os
directory = "/Users/jfeng1/Datasets/Kinase/proasis_biib_all"
pdbs = glob.glob(os.path.join(directory,"*.ent.gz"))
ref = os.path.join(directory,"receptor.pdb")
ref_lig = os.path.join(directory,"ligand_frag.sdf")

In [26]:
ifs = oemolistream()
ifs.open(ref)
ref_mol = OEGraphMol()
OEReadMolecule(ifs,ref_mol)
ifs.close()

ifs = oemolistream()
ifs.open(ref_lig)
lig_mol = OEGraphMol()
OEReadMolecule(ifs,lig_mol)
ifs.close()
ov = OEOverlap()
ov.SetMethod(OEOverlapMethod_Analytic)
ov.SetRefMol(lig_mol)

target_lig = OEGraphMol()
target_receptor = OEGraphMol()
target_wat = OEGraphMol()
other = OEGraphMol()

for pdb in pdbs:
    ifs = oemolistream()
    ifs.open(pdb)
    target_mol = OEGraphMol()
    OEReadMolecule(ifs,target_mol)
    ifs.close()

    rot = OEDoubleArray(9)
    trans = OEDoubleArray(3)
    alignment = OEGetAlignment(ref_mol, target_mol)
    rmsd = OERMSD(ref_mol,target_mol,alignment,True,True, rot, trans)
    
    OERotate(target_mol,rot)
    OETranslate(target_mol,trans)
    if OESplitMolComplex(target_lig,target_receptor,target_wat,other,target_mol):
        print target_lig.NumAtoms()
        if target_lig.NumAtoms() > 0 and rmsd < 10:
            rs = OEOverlapResults()
            ov.Overlap(target_lig,rs)
            print rs.tanimoto
            if rs.tanimoto > 0.2:
                print rs.tanimoto,pdb
                ofs = oemolostream()
                filename1 = os.path.splitext(os.path.basename(pdb))[0]
                filename2 = "%s_rmsd.pdb" % (filename1)
                filename3 = os.path.join(directory,filename2)
                ofs.open(filename3)
                OEWriteMolecule(ofs,target_mol)
                ofs.close()
        
    


31
0.0108021385968
0.0108021385968 /Users/jfeng1/Datasets/Kinase/proasis_biib/3d0e.pdb.gz
